In [1]:
from grid2op.Agent import BaseAgent
from grid2op.Converter.IdToAct import IdToAct



from grid2op.multi_agent.ma_typing import LocalObservation, LocalObservationSpace, \
    LocalAction, LocalActionSpace 

from grid2op import make
from grid2op.Action.PlayableAction import PlayableAction
from grid2op.Action import BaseAction
from grid2op.multi_agent.multiAgentEnv import MultiAgentEnv
import numpy as np
from grid2op.multi_agent.multi_agentExceptions import *

    
import numpy as np

from lightsim2grid import LightSimBackend
bk_cls = LightSimBackend

action_domains = {
    'agent_0' : [0,1,2,3, 4],
    'agent_1' : [5,6,7,8,9,10,11,12,13]
}
#env_name = "l2rpn_case14_sandbox"#"educ_case14_storage"
#env = make(env_name, test=False, backend = bk_cls(),
#                action_class=PlayableAction, _add_to_name="_test_ma")
#
#
#ma_env = MultiAgentEnv(env, action_domains, copy_env=False)
#
#ma_env.seed(0)
#obs = ma_env.reset()

### Env wrapper

In [3]:
def _get_tested_action(action_space):
    res = [action_space({})]  # add the do nothing
    # better use "get_all_unitary_topologies_set" and not "get_all_unitary_topologies_change"
    # maybe "change" are still "bugged" (in the sens they don't count all topologies exactly once)
    res += action_space.get_all_unitary_topologies_set(action_space)
    return res

#ALL_ACTIONS_SINGLE = _get_tested_action(env.action_space)

In [18]:
import gym
from gym.spaces import Discrete, Box

class EnvWrapper(gym.Env):
    def __init__(self, env_config=None):
        env_name = "l2rpn_case14_sandbox"#"educ_case14_storage"
        self.env = make(env_name, test=False, backend = bk_cls(),
                        action_class=PlayableAction, _add_to_name="_test_ma")
        self.env.seed(0)
        self.all_actions = self._get_tested_action(self.env.action_space)
        obs = self.env.reset().to_vect()
        
        self.observation_space = Box(shape=obs.shape, high=np.infty, low=-np.infty)#GymObservationSpace(self.env)
        self.action_space = Discrete(len(self.all_actions))
        
        self.reward_range = self.env.reward_range
        
        
    def reset(self):
        obs = self.env.reset()
        return obs.to_vect()
    
    def step(self, action):
        a = self.all_actions[action]
        
        obs, r, info, done = self.env.step(a)
        
        return obs.to_vect(), r, done, info
    
    def _get_tested_action(self, action_space):
        res = [action_space({})]  # add the do nothing
        # better use "get_all_unitary_topologies_set" and not "get_all_unitary_topologies_change"
        # maybe "change" are still "bugged" (in the sens they don't count all topologies exactly once)
        res += action_space.get_all_unitary_topologies_set(action_space)
        return res
    

from ray.rllib.env.multi_agent_env import MultiAgentEnv as MAEnv

class MAEnvWrapper(MAEnv):
    def __init__(self, env_config=None):
        super().__init__
        
        env_name = "l2rpn_case14_sandbox"#"educ_case14_storage"
        env = make(env_name, test=False, backend = bk_cls(),
                action_class=PlayableAction, _add_to_name="_test_ma")


        self.ma_env = MultiAgentEnv(env, action_domains, copy_env=False)
        self._agent_ids = set(self.ma_env.agents)
        self.ma_env.seed(0)
        obs = self.ma_env.reset()[self.ma_env.agents[0]].to_vect()
        self.observation_space = Box(shape=obs.shape, high=np.infty, low=-np.infty)#{
        #    agent : Box(shape=obs.shape, high=np.infty, low=-np.infty)
        #    for agent in self.ma_env.agents
        #}
        
        self.all_actions = {
            agent : self._get_tested_action(self.ma_env.action_spaces[agent])
            for agent in self.ma_env.agents
        }
        self.action_space = {
            agent : Discrete(len(self.all_actions[agent]))
            for agent in self.ma_env.agents
        }
        
    def reset(self):
        obs = self.ma_env.reset()
        o = obs[self.ma_env.agents[0]].to_vect()
        return {
            agent : o.copy()
            for agent in self.ma_env.agents
        }
    
    def step(self, actions):
        a = {
            agent : self.all_actions[agent][actions[agent]]
            for agent in self.ma_env.agents
        }
        
        obs, r, info, done = self.ma_env.step(a)
        done['__all__'] = done[self.ma_env.agents[0]]
        #info['__all__'] = ''
        o = obs[self.ma_env.agents[0]].to_vect()
        obs = {
            agent : o.copy()
            for agent in self.ma_env.agents
        }
        info = dict()
        return obs, r, done, info
    
    def _get_tested_action(self, action_space):
        res = [action_space({})]  # add the do nothing
        # better use "get_all_unitary_topologies_set" and not "get_all_unitary_topologies_change"
        # maybe "change" are still "bugged" (in the sens they don't count all topologies exactly once)
        res += action_space.get_all_unitary_topologies_set(action_space)
        return res

In [14]:
new_env = EnvWrapper()
new_ma_env = MAEnvWrapper()

/home/najarfar/Internship/Grid2Op/grid2op/multi_agent/multiAgentEnv.py:123: UserWarning: Rules can not be changed in this version.
  warnings.warn("Rules can not be changed in this version.")
/home/najarfar/Internship/Grid2Op/grid2op/multi_agent/multiAgentEnv.py:129: UserWarning: The central env has been heavily modified (parameters and reset) !
  warnings.warn("The central env has been heavily modified (parameters and reset) !")


## RL agents

In [41]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

model = PPO("MlpPolicy", new_env, verbose=1)
model.learn(total_timesteps=250)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 3.55     |
|    ep_rew_mean     | 153      |
| time/              |          |
|    fps             | 28       |
|    iterations      | 1        |
|    time_elapsed    | 71       |
|    total_timesteps | 2048     |
---------------------------------


In [4]:
import ray
from ray.rllib.agents.ppo import ppo

import pandas as pd
import json
import os
import shutil
import sys

checkpoint_root = "./single_ppo"
# Where checkpoints are written:
shutil.rmtree(checkpoint_root, ignore_errors=True, onerror=None)

# Where some data will be written and used by Tensorboard below:
ray_results = f'{os.getenv("HOME")}/ray_results/'
shutil.rmtree(ray_results, ignore_errors=True, onerror=None)

info = ray.init(ignore_reinit_error=True)
print("Dashboard URL: http://{}".format(info["webui_url"]))


Dashboard URL: http://


/tmp/ipykernel_2215429/1968686887.py:19: DeprecationWarning: Accessing values through ctx["webui_url"] is deprecated. Use ctx.address_info["webui_url"] instead.
  print("Dashboard URL: http://{}".format(info["webui_url"]))


In [9]:
SELECT_ENV = EnvWrapper                            # Specifies the OpenAI Gym environment for Cart Pole
N_ITER = 10                                     # Number of training runs.

config = ppo.DEFAULT_CONFIG.copy()              # PPO's default configuration. See the next code cell.
config["log_level"] = "WARN"                    # Suppress too many messages, but try "INFO" to see what can be printed.

# Other settings we might adjust:
config["num_workers"] = 1                       # Use > 1 for using more CPU cores, including over a cluster
config["num_sgd_iter"] = 10                     # Number of SGD (stochastic gradient descent) iterations per training minibatch.
                                                # I.e., for each minibatch of data, do this many passes over it to train. 
config["sgd_minibatch_size"] = 64              # The amount of data records per minibatch
config["model"]["fcnet_hiddens"] = [100, 50]    #
config["num_cpus_per_worker"] = 0  # This avoids running out of resources in the notebook environment when this cell is re-executed

#config['disable_env_checking']=True

In [10]:
agent = ppo.PPOTrainer(config, env=SELECT_ENV)

results = []
episode_data = []
episode_json = []

for n in range(N_ITER):
    result = agent.train()
    results.append(result)
    
    episode = {'n': n, 
               'episode_reward_min': result['episode_reward_min'], 
               'episode_reward_mean': result['episode_reward_mean'], 
               'episode_reward_max': result['episode_reward_max'],  
               'episode_len_mean': result['episode_len_mean']}
    
    episode_data.append(episode)
    episode_json.append(json.dumps(episode))
    file_name = agent.save(checkpoint_root)
    
    print(f'{n:3d}: Min/Mean/Max reward: {result["episode_reward_min"]:8.4f}/{result["episode_reward_mean"]:8.4f}/{result["episode_reward_max"]:8.4f}. Checkpoint saved to {file_name}')


2022-08-23 10:05:34,936	INFO trainer.py:2332 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2022-08-23 10:05:34,938	INFO ppo.py:414 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2022-08-23 10:05:34,939	INFO trainer.py:903 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(RolloutWorker pid=2210590) 2022-08-23 10:05:44,346	WARNING env.py:135 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
2022-08-23 10:05:46,170	

  0: Min/Mean/Max reward: -10.0000/153.8404/844.6186. Checkpoint saved to ./single_ppo/cart/checkpoint_000001/checkpoint-1
  1: Min/Mean/Max reward: -10.0000/179.9672/1109.2118. Checkpoint saved to ./single_ppo/cart/checkpoint_000002/checkpoint-2
  2: Min/Mean/Max reward: -10.0000/205.8721/996.2866. Checkpoint saved to ./single_ppo/cart/checkpoint_000003/checkpoint-3
  3: Min/Mean/Max reward: -10.0000/238.8013/1270.9819. Checkpoint saved to ./single_ppo/cart/checkpoint_000004/checkpoint-4
  4: Min/Mean/Max reward: -10.0000/267.5639/1759.7718. Checkpoint saved to ./single_ppo/cart/checkpoint_000005/checkpoint-5
  5: Min/Mean/Max reward: -10.0000/301.4468/1499.2796. Checkpoint saved to ./single_ppo/cart/checkpoint_000006/checkpoint-6
  6: Min/Mean/Max reward: -10.0000/336.7569/1938.4094. Checkpoint saved to ./single_ppo/cart/checkpoint_000007/checkpoint-7
  7: Min/Mean/Max reward: -10.0000/363.0247/2188.7150. Checkpoint saved to ./single_ppo/cart/checkpoint_000008/checkpoint-8
  8: Min/M

In [14]:
with open('single_ppo/rewards.json', 'w') as outfile:
    json.dump(episode_json, outfile)

In [19]:
import ray
from ray.rllib.agents.ppo import ppo

import pandas as pd
import json
import os
import shutil
import sys

checkpoint_root = "./ma_ppo"
# Where checkpoints are written:
shutil.rmtree(checkpoint_root, ignore_errors=True, onerror=None)

# Where some data will be written and used by Tensorboard below:
ray_results = f'{os.getenv("HOME")}/ray_results/'
shutil.rmtree(ray_results, ignore_errors=True, onerror=None)

info = ray.init(ignore_reinit_error=True)
print("Dashboard URL: http://{}".format(info["webui_url"]))

#Configs
SELECT_ENV = MAEnvWrapper                            # Specifies the OpenAI Gym environment for Cart Pole
N_ITER = 10                                     # Number of training runs.

config = ppo.DEFAULT_CONFIG.copy()              # PPO's default configuration. See the next code cell.
config["log_level"] = "WARN"                    # Suppress too many messages, but try "INFO" to see what can be printed.

# Other settings we might adjust:
config["num_workers"] = 1                       # Use > 1 for using more CPU cores, including over a cluster
config["num_sgd_iter"] = 10                     # Number of SGD (stochastic gradient descent) iterations per training minibatch.
                                                # I.e., for each minibatch of data, do this many passes over it to train. 
config["sgd_minibatch_size"] = 64              # The amount of data records per minibatch
config["model"]["fcnet_hiddens"] = [100, 50]    #
config["num_cpus_per_worker"] = 0  # This avoids running out of resources in the notebook environment when this cell is re-executed

from ray.rllib.policy.policy import PolicySpec
config["multiagent"] = {
    "policies" : {
        "agent_0" : PolicySpec(
            action_space=Discrete(len(new_ma_env.all_actions["agent_0"]))
        ),
        "agent_1" : PolicySpec(
            action_space=Discrete(len(new_ma_env.all_actions["agent_1"]))
        )
    },
    "policy_mapping_fn":
            lambda agent_id, episode, worker, **kwargs : agent_id
}

#Trainer
agent = ppo.PPOTrainer(config, env=SELECT_ENV)

results = []
episode_data = []
episode_json = []

for n in range(N_ITER):
    result = agent.train()
    results.append(result)
    
    episode = {'n': n, 
               'episode_reward_min': result['episode_reward_min'], 
               'episode_reward_mean': result['episode_reward_mean'], 
               'episode_reward_max': result['episode_reward_max'],  
               'episode_len_mean': result['episode_len_mean']}
    
    episode_data.append(episode)
    episode_json.append(json.dumps(episode))
    file_name = agent.save(checkpoint_root)
    
    print(f'{n:3d}: Min/Mean/Max reward: {result["episode_reward_min"]:8.4f}/{result["episode_reward_mean"]:8.4f}/{result["episode_reward_max"]:8.4f}. Checkpoint saved to {file_name}')

    with open('ma_ppo/rewards.json', 'w') as outfile:
        json.dump(episode_json, outfile)

2022-08-23 11:32:04,924	INFO worker.py:973 -- Calling ray.init() again after it has already been called.


Dashboard URL: http://


(RolloutWorker pid=2218030) /home/najarfar/Internship/Grid2Op/grid2op/multi_agent/multiAgentEnv.py:123: UserWarning: Rules can not be changed in this version.
(RolloutWorker pid=2218030)   warnings.warn("Rules can not be changed in this version.")
(RolloutWorker pid=2218030) /home/najarfar/Internship/Grid2Op/grid2op/multi_agent/multiAgentEnv.py:129: UserWarning: The central env has been heavily modified (parameters and reset) !
(RolloutWorker pid=2218030)   warnings.warn("The central env has been heavily modified (parameters and reset) !")
(RolloutWorker pid=2218030) 2022-08-23 11:32:14,341	WARNING env.py:216 -- Your MultiAgentEnv <MAEnvWrapper instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__` from within your MutiAgentEnv's constructor. This will raise an error in the future.
2022-08-23 11:32:17,379	INFO trainable.py:159 -- Trainable.setup took 12.438 seconds. If your trainable is slow to initialize, consider setting reuse_a

FileNotFoundError: [Errno 2] No such file or directory: '/home/najarfar/ray_results/PPOTrainer_MAEnvWrapper_2022-08-23_11-32-04wbk31c1q/progress.csv'

: 